In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import pandas as pd
import numpy as np
import tabula
import os

***DATA EXTRACTION***

In [2]:
# WEB SCRAPPING TABLE (df_salario):

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service = service)
driver.get('https://portal.trt3.jus.br/internet/servicos/valores/salario-minimo')
url = driver.find_element(By.XPATH,'//*[@id="parent-fieldname-text"]/div/table')
html = url.get_attribute('outerHTML')
driver.quit()
df_salario = pd.read_html(html)[0]

In [3]:
# WEB SCRAPPING TABLE FROM PDF (margens_2013):

margens_2013 = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2013.pdf', pages = 'all')
margens_13 = pd.concat(margens_2013)

In [4]:
# WEB SCRAPPING TABLE FROM PDF (margens_2023):

margens = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2023.pdf', pages = 'all')
margens_23 = pd.concat(margens)

In [5]:
#READING XLSX FILES IN DATAFRAME (brasil):

brasil = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\brasil_semanal.xlsx')

In [6]:
#READING XLSX FILES IN DATAFRAME (estados):

estados = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\estados_semanal.xlsx')

In [7]:
#READING XLSX FILES IN DATAFRAME COMPLEMENT STATES ICMS ()
to = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\TO.xlsx')
sc = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\SC.xlsx')
rn = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\RN.xlsx')
pr = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\PR.xlsx')
ms = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\MS.xlsx')
es = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\ES.xlsx')
am = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\AM.xlsx')

complemento_icms = pd.concat([to,sc,rn,pr,ms,es,am])

***EXPLORING AND CLEANING THE DATA***

*DATAFRAME df_salario:*

In [8]:
#EXPLORING THE df_salario DATAFRAME:

df_salario.head()

,ANO/VIGÊNCIA,VALOR,MOEDA,FONTE
0,2023 1º/1/2023,"R$ 1.302,00 Por Mês R$ 43,40 Por Dia R$ 5,92 P...",Real,"MP n. 1.143, de 12/12/2022 DOU 12/12/2022 - Ed..."
1,- 2022 1º/1/2022,"R$ 1.212,00 Por Mês R$ 40,40 Por Dia R$ 5,51 P...",Real,"MP n. 1.091, de 30/12/2021 DOU 31/12/2021 (Con..."
2,2021 1º/1/2021,"R$ 1.100,00 Por Mês R$ 36,67 Por Dia R$ 5,00 P...",Real,"MP n. 1.021, de 30/12/2020 DOU 31/12/20 (Conve..."
3,2020 1º/2/2020,"R$ 1.045,00 Por Mês R$ 34,83 Por Dia R$ 4,75 P...",Real,"MP n. 919, de 30/1/2020 DOU 31/1/20"
4,2020 1º/1/2020,"R$ 1.039,00 Por Mês R$ 34,63 Por Dia R$ 4,72 P...",Real,"MP n. 916, de 31/12/2019 DOU 31/12/19 - Edição..."


In [9]:
#EXPLORING THE df_salario DATAFRAME:

df_salario.tail()

,ANO/VIGÊNCIA,VALOR,MOEDA,FONTE
129,1954 4/7/1954,"Cr$ 2.200,00 Por Mês (valor regional para BH e...",Cruzeiro,"Decreto n.35.450, de 1º/5/1954 DOU 4/5/54"
130,1952 1º/1/1952,"Cr$ 900,00 Por Mês Cr$ 30,00 Por Dia Cr$ 3,75 ...",Cruzeiro,"Decreto n.30.342, de 24/12/1951 DOU 26/12/51"
131,1943 1º/12/1943,"Cr$ 270,00 Por Mês Cr$ 10,80 Por Dia Cr$ 1,35 ...",Cruzeiro,"Decreto-Lei n. 5.977, de 10/11/1943 DOU 22/11/43"
132,1943 17/7/1943,"Cr$ 212,00 Por Mês Cr$ 8,48 Por Dia Cr$ 1,06 P...",Cruzeiro,"Decreto-Lein.5.670, de 15/7/1943 DOU 17/7/43"
133,1940 4/7/1940,170$000 Por Mês 6$800 Por Dia $850 Por Hora (v...,Real,"Decreto-Lein.2.162, de 1º/5/1940 DOU 4/5/40 (D..."


In [10]:
#EXPLORING THE df_salario DATAFRAME:

df_salario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ANO/VIGÊNCIA  134 non-null    object
 1   VALOR         134 non-null    object
 2   MOEDA         134 non-null    object
 3   FONTE         134 non-null    object
dtypes: object(4)
memory usage: 4.3+ KB


In [11]:
#PERFORMING TREATMENT IN DATAFRAME df_salario USING DE FUNCTION BELOW:
    
def limpar_df_salario(df):
    
    '''THIS FUNCTION PERFORMS THE CLEANING TREATMENT ON THE DATAFRAME 'df_salario'.'''
    
    df = df.drop(columns = ['MOEDA','FONTE']).drop(index=range(12,134))
    df = df.rename(columns = {'ANO/VIGÊNCIA':'DATA', 'VALOR':'VALOR POR MÊS'})
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.slice(stop=11)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace('P','').str.replace(',','.')
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'R\$ +', '', regex=True)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'R\$\xa0', '', regex=True)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'\xa0', '', regex=True)
    df['DATA'] = df['DATA'].str.replace(r'\s+\d{1,2}º/\d{1,2}/\d{4}', '', regex=True).str.replace('- ','')
    df = df.set_index('DATA')
    
    return df


In [12]:
# NEW DATAFRAME TREATED (salario_minimo):

salario_minimo = limpar_df_salario(df_salario)

*dataframe margens_2013*

In [13]:
#EXPLORING THE margens_2013 DATAFRAME:

margens_13.head()

,Unnamed: 0,Unnamed: 1,jan/13,fev/13,mar/13,abr/13,mai/13,jun/13,Unnamed: 2,jul/13,...,out/13,Unnamed: 4,nov/13,Unnamed: 5,dez/13,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,"Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea ""b"" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa ""Auxílio-Gás"" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa ""Bolsa-Família"" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas."
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brasil,Preço de Realização do Produtor,"11,33","11,34","11,34","11,34","11,34","11,34",NaN,"11,34",...,"11,34",NaN,"11,34",NaN,"11,33",NaN,NaN,NaN,NaN,NaN
2,NaN,CIDE,-,-,-,-,-,-,NaN,-,...,-,NaN,-,NaN,-,NaN,NaN,NaN,NaN,NaN
3,NaN,PIS/COFINS,"2,18","2,18","2,18","2,18","2,18","2,18",NaN,"2,18",...,"2,18",NaN,"2,18",NaN,"2,18",NaN,NaN,NaN,NaN,NaN
4,NaN,Preço do Produtor s/ ICMS c/ CIDE/PIS/COFINS,"13,51","13,52","13,52","13,52","13,52","13,52",NaN,"13,52",...,"13,52",NaN,"13,52",NaN,"13,51",NaN,NaN,NaN,NaN,NaN


In [14]:
#EXPLORING THE margens_2013 DATAFRAME:

margens_13.tail()

,Unnamed: 0,Unnamed: 1,jan/13,fev/13,mar/13,abr/13,mai/13,jun/13,Unnamed: 2,jul/13,...,out/13,Unnamed: 4,nov/13,Unnamed: 5,dez/13,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,"Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea ""b"" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa ""Auxílio-Gás"" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa ""Bolsa-Família"" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas."
2,Roraima\r(alíquota de ICMS\rdo produtor)\r17%,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"10,81\r-\r2,18\r12,99\r182,06%\r184,11%\r7,53\...","10,85\r-\r2,18\r13,03\r183,52%\r180,76%\r7,53\...","10,78\r-\r2,18\r12,96\r190,67%\r191,64%\r7,73\...","10,88\r-\r2,18\r13,06\r188,84%\r189,00%\r7,73\...","10,79\r-\r2,18\r12,97\r190,74%\r190,79%\r7,73\...","10,83\r-\r2,18\r13,01\r189,49%\r190,31%\r7,73\...",NaN,"10,80\r-\r2,18\r12,98\r190,09%\r190,97%\r7,73\...",...,"10,81\r-\r2,18\r12,99\r190,21%\r190,79%\r7,73\...",NaN,"10,86\r-\r2,18\r13,04\r188,23%\r190,06%\r7,73\...",NaN,"10,83\r-\r2,18\r13,01\r189,43%\r190,26%\r7,73\...",NaN,NaN,NaN,NaN,NaN
0,Santa Catarina\r(alíquota de ICMS\rdo produtor...,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",...,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,NaN,NaN,NaN,NaN
1,São Paulo\r(alíquota de ICMS\rdo pr

In [15]:
#EXPLORING THE margens_2013 DATAFRAME:

margens_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 0 to 0
Data columns (total 23 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [16]:
#PERFORMING TREATMENT IN DATAFRAME df_salario USING DE FUNCTION BELOW:

def brasil_icms(margens_13):
    
    estado = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    estado = estado.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    estado = estado.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    estado = estado.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    estado = estado.iloc[0:2]
    estado = estado.reset_index(drop=True).drop(index=[0])
    estado.fillna('BRASIL', inplace=True)
    return estado



In [17]:
brasil_icms(margens_13)

,ESTADO,ICMS,jan/13,fev/13,mar/13,abr/13,mai/13,jun/13,jul/13,ago/13,set/13,out/13,nov/13,dez/13
1,BRASIL,ICMS,"4,84","4,85","4,87","4,98","5,00","5,00","5,00","5,00","5,01","5,04","5,04","5,05"


*DATAFRAME brasil*

In [18]:
#EXPLORING THE brasil DATAFRAME:
brasil.head()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,SUPERINTENDÊNCIA DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SISTEMA DE LEVANTAMENTO DE PREÇOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RELATÓRIO DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#EXPLORING THE brasil DATAFRAME:
brasil.tail()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
3295,2023-03-26 00:00:00,2023-04-01 00:00:00,GASOLINA COMUM,4312,R$/l,5.48,0.35,4.27,7.19,-,0.064,-,-,-,-,-
3296,2023-03-26 00:00:00,2023-04-01 00:00:00,GLP,3944,R$/13kg,107.33,11.693,74.99,155,-,0.109,-,-,-,-,-
3297,2023-03-26 00:00:00,2023-04-01 00:00:00,GNV,353,R$/m³,4.5,0.543,3.54,6.59,-,0.12,-,-,-,-,-
3298,2023-03-26 00:00:00,2023-04-01 00:00:00,OLEO DIESEL,1953,R$/l,5.8,0.397,5.06,7.69,-,0.068,-,-,-,-,-
3299,2023-03-26 00:00:00,2023-04-01 00:00:00,OLEO DIESEL S10,3104,R$/l,5.86,0.391,5.09,8.49,-,0.067,-,-,-,-,-


In [20]:
#EXPLORING THE brasil DATAFRAME:
brasil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3300 entries, 0 to 3299
Data columns (total 16 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP  3297 non-null   object
 1   Unnamed: 1                                                         3284 non-null   object
 2   Unnamed: 2                                                         3284 non-null   object
 3   Unnamed: 3                                                         3284 non-null   object
 4   Unnamed: 4                                                         3284 non-null   object
 5   Unnamed: 5                                                         3284 non-null   object
 6   Unnamed: 6                                                         3284 non-null   object
 7   Unnamed: 7                       

In [21]:
#PERFORMING TREATMENT IN DATAFRAME brasil USING DE FUNCTION BELOW:

def limpar_df_br(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime
    df = df.rename(columns = {'AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP':'DATA INICIAL','Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'PRODUTO','Unnamed: 3':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 4':'UNIDADE DE MEDIDA','Unnamed: 5':'PREÇO MÉDIO REVENDA','Unnamed: 6':'DESVIO PADRÃO REVENDA','Unnamed: 7':'PREÇO MÍNIMO REVENDA','Unnamed: 8':'PREÇO MÁXIMO REVENDA','Unnamed: 9':'MARGEM MÉDIA REVENDA','Unnamed: 10':'COEF DE VARIAÇÃO REVENDA','Unnamed: 11':'PREÇO MÉDIO DISTRIBUIÇÃO','Unnamed: 12':'DESVIO PADRÃO DISTRIBUIÇÃO','Unnamed: 13':'PREÇO MÍNIMO DISTRIBUIÇÃO','Unnamed: 14':'PREÇO MÁXIMO DISTRIBUIÇÃO','Unnamed: 15':'COEF DE VARIAÇÃO DISTRIBUIÇÃO'})
    df_1 = df.drop(index=range(0, 17)).drop(columns=['DATA INICIAL', 'DESVIO PADRÃO REVENDA', 'MARGEM MÉDIA REVENDA','COEF DE VARIAÇÃO REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO','DESVIO PADRÃO DISTRIBUIÇÃO',	'PREÇO MÍNIMO DISTRIBUIÇÃO','PREÇO MÁXIMO DISTRIBUIÇÃO','COEF DE VARIAÇÃO DISTRIBUIÇÃO'])
    df_limpo = df_1.loc[df_1['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo.index.name = 'DATA'
    return df_limpo


In [22]:
# NEW DATAFRAME TREATED (glp_br):
glp_br = limpar_df_br(brasil)

***DATAFRAME estado***

In [23]:
#EXPLORING THE (estados) DATAFRAME:
estados.head()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,SUPERINTENDÊNCIA DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SISTEMA DE LEVANTAMENTO DE PREÇOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RELATÓRIO DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#EXPLORING THE (estados) DATAFRAME:
estados.tail()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
82865,2023-03-26 00:00:00,2023-04-01 00:00:00,NORTE,RORAIMA,OLEO DIESEL S10,14,R$/l,7,0.041,6.97,7.13,-,0.006,-,-,-,-,-
82866,2023-03-26 00:00:00,2023-04-01 00:00:00,SUL,SANTA CATARINA,OLEO DIESEL S10,133,R$/l,5.86,0.261,5.31,6.79,-,0.045,-,-,-,-,-
82867,2023-03-26 00:00:00,2023-04-01 00:00:00,SUDESTE,SAO PAULO,OLEO DIESEL S10,785,R$/l,5.86,0.402,5.09,8.49,-,0.069,-,-,-,-,-
82868,2023-03-26 00:00:00,2023-04-01 00:00:00,NORDESTE,SERGIPE,OLEO DIESEL S10,24,R$/l,5.87,0.303,5.57,6.49,-,0.052,-,-,-,-,-
82869,2023-03-26 00:00:00,2023-04-01 00:00:00,NORTE,TOCANTINS,OLEO DIESEL S10,41,R$/l,5.61,0.307,5.25,6.9,-,0.055,-,-,-,-,-


In [25]:
#EXPLORING THE (estados) DATAFRAME:
estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82870 entries, 0 to 82869
Data columns (total 18 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP  82867 non-null  object
 1   Unnamed: 1                                                         82854 non-null  object
 2   Unnamed: 2                                                         82854 non-null  object
 3   Unnamed: 3                                                         82854 non-null  object
 4   Unnamed: 4                                                         82854 non-null  object
 5   Unnamed: 5                                                         82854 non-null  object
 6   Unnamed: 6                                                         82854 non-null  object
 7   Unnamed: 7                     

In [26]:
#PERFORMING TREATMENT IN DATAFRAME estados USING DE FUNCTION BELOW:

def limpar_df_estado(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime   
    df = df.drop(index=range(0, 17)).drop(columns = ['AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP', 'Unnamed: 8', 'Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17'])
    estados_limpo = df.rename(columns = {'Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'REGIÃO','Unnamed: 3':'ESTADO','Unnamed: 4':'PRODUTO','Unnamed: 5':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 6':'UNIDADE DE MEDIDA','Unnamed: 7':'PREÇO MÉDIO REVENDA','Unnamed: 9':'PREÇO MÍNIMO REVENDA','Unnamed: 10':'PREÇO MÁXIMO REVENDA'})
    df_limpo = estados_limpo.loc[estados_limpo['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo = df_limpo.set_index('DATA FINAL')
    df_limpo.index.name = 'DATA'
    return df_limpo


In [27]:
# NEW DATAFRAME TREATED (glp_regioes):
glp_regioes = limpar_df_estado(estados)

***DATAFRAME margens_13***

In [28]:
#EXPLORING THE (margens_13) DATAFRAME:
margens_13.head()

,Unnamed: 0,Unnamed: 1,jan/13,fev/13,mar/13,abr/13,mai/13,jun/13,Unnamed: 2,jul/13,...,out/13,Unnamed: 4,nov/13,Unnamed: 5,dez/13,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,"Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea ""b"" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa ""Auxílio-Gás"" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa ""Bolsa-Família"" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas."
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brasil,Preço de Realização do Produtor,"11,33","11,34","11,34","11,34","11,34","11,34",NaN,"11,34",...,"11,34",NaN,"11,34",NaN,"11,33",NaN,NaN,NaN,NaN,NaN
2,NaN,CIDE,-,-,-,-,-,-,NaN,-,...,-,NaN,-,NaN,-,NaN,NaN,NaN,NaN,NaN
3,NaN,PIS/COFINS,"2,18","2,18","2,18","2,18","2,18","2,18",NaN,"2,18",...,"2,18",NaN,"2,18",NaN,"2,18",NaN,NaN,NaN,NaN,NaN
4,NaN,Preço do Produtor s/ ICMS c/ CIDE/PIS/COFINS,"13,51","13,52","13,52","13,52","13,52","13,52",NaN,"13,52",...,"13,52",NaN,"13,52",NaN,"13,51",NaN,NaN,NaN,NaN,NaN


In [29]:
#EXPLORING THE (margens_13) DATAFRAME:
margens_13.tail()

,Unnamed: 0,Unnamed: 1,jan/13,fev/13,mar/13,abr/13,mai/13,jun/13,Unnamed: 2,jul/13,...,out/13,Unnamed: 4,nov/13,Unnamed: 5,dez/13,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,"Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea ""b"" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa ""Auxílio-Gás"" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa ""Bolsa-Família"" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas."
2,Roraima\r(alíquota de ICMS\rdo produtor)\r17%,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"10,81\r-\r2,18\r12,99\r182,06%\r184,11%\r7,53\...","10,85\r-\r2,18\r13,03\r183,52%\r180,76%\r7,53\...","10,78\r-\r2,18\r12,96\r190,67%\r191,64%\r7,73\...","10,88\r-\r2,18\r13,06\r188,84%\r189,00%\r7,73\...","10,79\r-\r2,18\r12,97\r190,74%\r190,79%\r7,73\...","10,83\r-\r2,18\r13,01\r189,49%\r190,31%\r7,73\...",NaN,"10,80\r-\r2,18\r12,98\r190,09%\r190,97%\r7,73\...",...,"10,81\r-\r2,18\r12,99\r190,21%\r190,79%\r7,73\...",NaN,"10,86\r-\r2,18\r13,04\r188,23%\r190,06%\r7,73\...",NaN,"10,83\r-\r2,18\r13,01\r189,43%\r190,26%\r7,73\...",NaN,NaN,NaN,NaN,NaN
0,Santa Catarina\r(alíquota de ICMS\rdo produtor...,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",...,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,NaN,NaN,NaN,NaN
1,São Paulo\r(alíquota de ICMS\rdo pr

In [30]:
#EXPLORING THE (margens_13) DATAFRAME:
margens_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 0 to 0
Data columns (total 23 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [31]:
# DEFINING THE FUNCTION THAT EXTRACTS BRAZIL ONLY FROM THE GROUP.
def brasil_icms(margens_13, estado):
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE 'margens_13' DATAFRAME, AIMED AT RETURNING GENERAL DATA FOR BRAZIL.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.replace('ICMS', '-')
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS em %'	})
    df = df.iloc[0:2]
    df = df.reset_index(drop=True).drop(index=[0])
    df.fillna(estado, inplace=True)
    df = df.replace('ICMS', '-')
    df = df.set_index('ESTADO')
    
    return df

In [32]:
 # NEW DATAFRAME TREATED (brasil_icms_13):
 
 brasil_icms_13 = brasil_icms(margens_13, 'BRASIL' ) 

In [33]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:
def acre_icms_13(margens_13, estado):
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[2:12]
    df = df.drop(index = [2,3,4,6,7,8,9,10,11])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df    

In [34]:
 # NEW DATAFRAME TREATED (ac_13):

ac_13 = acre_icms_13(margens_13, 'AC')

In [35]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def alagoas_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[13:21]
    df = df.drop(index = [13,14,15,17,18,19,20])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
   
    return df

In [36]:
 # NEW DATAFRAME TREATED (al_13):
 
al_13 = alagoas_icms_13(margens_13,'AL')

In [37]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def ceara_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[27:38]
    df = df.drop(index = [27,28,29,31,32,33,34,35,36,37])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [38]:
# NEW DATAFRAME TREATED (ce_13):

ce_13 = ceara_icms_13(margens_13,'CE')

In [39]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def distrito_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[38:49]
    df = df.drop(index = [38,39,40,42,43,44,45,46,47,48])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('12%', estado).replace('ICMS','12%')
    df = df.set_index('ESTADO')
    
    return df

In [40]:
# NEW DATAFRAME TREATED (distrito_13):

distrito_13 = distrito_icms_13(margens_13,'DF')

In [41]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def piaui_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[69:80]
    df = df.drop(index = [69,70,71,73,74,75,76,77,78,79])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [42]:
# NEW DATAFRAME TREATED (pi_13):

pi_13 = piaui_icms_13(margens_13,'PI')

In [43]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def espirito_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[49:57]
    df = df.drop(index = [49,50,51,53,54,55,56])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [44]:
# NEW DATAFRAME TRATED (es_13):

es_13 = espirito_icms_13(margens_13,'ES')

In [45]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def rj_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[80:91]
    df = df.drop(index = [80,81,82,84,85,86,87,88,89,90])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('12%', estado).replace('ICMS','12%')
    df = df.set_index('ESTADO')
    
    return df

In [46]:
# NEW DATAFRAME TREATED (rj_13):

rj_13 = rj_icms_13(margens_13,'RJ')

In [47]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def rn_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''

    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[91:99]
    df = df.drop(index = [91,92,93,95,96,97,98])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('do produtor)', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [48]:
# NEW DATAFRAME TREATED (rn_13):

rn_13 = rn_icms_13(margens_13,'RN')

In [49]:
def limpar_meses_icms(df):
    '''THIS FUNCTION USES THE 'margins_13' DATAFRAME TO CLEAN AND PROCESS GENERAL DATA WITH THE PURPOSE OF EXTRACTING THE ICMS FROM EACH STATE.'''
    
# GENERAL TREATMENT:
    df = df.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS em %'})
    df = df.reset_index(drop=True)
    df = df.iloc[[21,22,23,57,58,59,60,61,62,63,64,65,99,100,101,102,103,104,105]]
    lista_estado = ['AP','AM','BA','GO','MA','MT','MS','MG','PA','PB','PR','PE','RS','RO','RR','SC','SP','SE','TO']
    lista_icms = ['12%','17%','12%','12%','17%','12%','12%','18%','17%','17%','12%','17%','12%','12%','17%','12%','12%','17%','12%']
    df['ICMS em %'] = lista_icms
    df['ESTADO'] = lista_estado

#'jan/13' COLUMN TREATMENT:

    df['jan/13'] = df['jan/13'].str.slice(start=33)
    df['jan/13'] = df['jan/13'].str.replace(r'%\r','',regex = True)
    df['jan/13'] = df['jan/13'].str.slice(start=0, stop=4)
    df['jan/13'] = df['jan/13'].str.replace('-\r17','-')
    df['jan/13'] = df['jan/13'].str.replace(',','.')

#'fev/13' COLUMN TREATMENT:

    df['fev/13'] = df['fev/13'].str.slice(start=33)
    df['fev/13'] = df['fev/13'].str.replace(r'%\r','',regex = True)
    df['fev/13'] = df['fev/13'].str.slice(start=0, stop=4)
    df['fev/13'] = df['fev/13'].str.replace('-\r21','-')
    df['fev/13'] = df['fev/13'].str.replace(',','.')


#'mar/13' COLUMN TREATMENT:

    df['mar/13'] = df['mar/13'].str.slice(start=33)
    df['mar/13'] = df['mar/13'].str.replace(r'%\r','',regex = True)
    df['mar/13'] = df['mar/13'].str.slice(start=0, stop=4)
    df['mar/13'] = df['mar/13'].str.replace('-\r22','-')
    df['mar/13'] = df['mar/13'].str.replace(',','.')


#'abr/13' COLUMN TREATMENT:

    df['abr/13'] = df['abr/13'].str.slice(start=33)
    df['abr/13'] = df['abr/13'].str.replace(r'%\r','',regex = True)
    df['abr/13'] = df['abr/13'].str.slice(start=0, stop=4)
    df['abr/13'] = df['abr/13'].str.replace(',','.')


#'mai/13' COLUMN TREATMENT:

    df['mai/13'] = df['mai/13'].str.slice(start=33)
    df['mai/13'] = df['mai/13'].str.replace(r'%\r','',regex = True)
    df['mai/13'] = df['mai/13'].str.slice(start=0, stop=4)
    df['mai/13'] = df['mai/13'].str.replace(',','.')


#'jun/13' COLUMN TREATMENT:

    df['jun/13'] = df['jun/13'].str.slice(start=33)
    df['jun/13'] = df['jun/13'].str.replace(r'%\r','',regex = True)
    df['jun/13'] = df['jun/13'].str.slice(start=0, stop=4)
    df['jun/13'] = df['jun/13'].str.replace(',','.')


#'jul/13' COLUMN TREATMENT:

    df['jul/13'] = df['jul/13'].str.slice(start=33)
    df['jul/13'] = df['jul/13'].str.replace(r'%\r','',regex = True)
    df['jul/13'] = df['jul/13'].str.slice(start=0, stop=4)
    df['jul/13'] = df['jul/13'].str.replace(',','.')


#'ago/13' COLUMN TREATMENT:

    df['ago/13'] = df['ago/13'].str.slice(start=33)
    df['ago/13'] = df['ago/13'].str.replace(r'%\r','',regex = True)
    df['ago/13'] = df['ago/13'].str.slice(start=0, stop=4)
    df['ago/13'] = df['ago/13'].str.replace(',','.')


#'set/13' COLUMN TREATMENT:

    df['set/13'] = df['set/13'].str.slice(start=33)
    df['set/13'] = df['set/13'].str.replace(r'%\r','',regex = True)
    df['set/13'] = df['set/13'].str.slice(start=0, stop=4)
    df['set/13'] = df['set/13'].str.replace(',','.')


#'out/13' COLUMN TREATMENT:

    df['out/13'] = df['out/13'].str.slice(start=33)
    df['out/13'] = df['out/13'].str.replace(r'%\r','',regex = True)
    df['out/13'] = df['out/13'].str.slice(start=0, stop=4)
    df['out/13'] = df['out/13'].str.replace(',','.')


#'nov/13' COLUMN TREATMENT:

    df['nov/13'] = df['nov/13'].str.slice(start=33)
    df['nov/13'] = df['nov/13'].str.replace(r'%\r','',regex = True)
    df['nov/13'] = df['nov/13'].str.slice(start=0, stop=4)
    df['nov/13'] = df['nov/13'].str.replace(',','.')


#'dez/13' COLUMN TREATMENT:

    df['dez/13'] = df['dez/13'].str.slice(start=33)
    df['dez/13'] = df['dez/13'].str.replace(r'%\r','',regex = True)
    df['dez/13'] = df['dez/13'].str.slice(start=0, stop=4)
    df['dez/13'] = df['dez/13'].str.replace(',','.')

# TRANSFORMING COLUMN 'ESTADO' IN INDEX:
    df = df.set_index('ESTADO')

    return df

In [50]:
# NEW DATAFRAME TREATED (faltantes_icms):

faltantes_icms = limpar_meses_icms(margens_13)

***CONCATENATING THE ICMS TABLES BY STATE IN A NEW DATAFRAME***

In [51]:
# NEW DATAFRAME TREATED (estados_icms_2013):

estados_icms_2013 = pd.concat([faltantes_icms, ac_13, al_13, ce_13, distrito_13, pi_13, es_13, rj_13, rn_13 ], axis=0)


***DATAFRAME complemento_icms***

In [52]:
#EXPLORING THE (complemento_icms) DATAFRAME:
complemento_icms.head()

,Column1,Column2,jan/23,fev/23
0,TO,0.12,10 .92,12 .01
0,SC,0.17,20.73,20.86
0,RN,0.18,16.58,16.58
0,PR,0.18,14.68,14.8
0,MS,0.12,8.86,8 .86


In [53]:
#EXPLORING THE (complemento_icms) DATAFRAME:
complemento_icms.tail()

,Column1,Column2,jan/23,fev/23
0,ES,0.17,13.68,14.76
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,\n,NaN
0,AM,0.18,15.4,15.6


In [54]:
#EXPLORING THE (complemento_icms) DATAFRAME:
complemento_icms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 0
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Column1  7 non-null      object 
 1   Column2  7 non-null      float64
 2   jan/23   8 non-null      object 
 3   fev/23   7 non-null      object 
dtypes: float64(1), object(3)
memory usage: 400.0+ bytes


In [55]:
def limpar_complemento(df):
    '''THIS FUNCTION USES THE 'complemento_icms' DATAFRAME TO CLEAN AND PROCESS GENERAL DATA WITH THE PURPOSE OF EXTRACTING THE ICMS FROM EACH STATE.'''
    df = df.rename(columns = {'Column1':'ESTADO','Column2':'ICMS em %'})
    df = df.reset_index(drop=True)
    df = df.drop(index=[6,7,8])
    jl = ['10.92', '20.73', '16.58', '14.68', '8.86', '13.68', '15.40']
    df['jan/23'] = jl
    jf = ['12.01', '20.86', '16.58', '14.80', '8.86', '14.76', '15.60']
    df['fev/23'] = jf
    pc =['12%','17%','18%','18%','12%', '17%', '18%']
    df['ICMS em %'] = pc
    df = df.set_index('ESTADO')
    
    return df

In [56]:
# NEW DATAFRAME TREATED (estados_complemento):

estados_complemento = limpar_complemento(complemento_icms)

***DATAFRAME margens_23***

In [57]:
#EXPLORING THE (margens_23) DATAFRAME:
margens_23.head()

,Unnamed: 0,Unnamed: 1,jan/23,fev/23
0,Brasil,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"42,84\r-\r-\r42,85\r12,95\r19,11\r74,89\r33,38...","43,03\r-\r-\r43,03\r13,17\r19,13\r75,33\r32,56..."
1,"Acre\r(alíquota de ICMS do produtor)\r17,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"48,31\r-\r-\r48,31 (2)\r16,09\r40,02\r104,41\...","47,22\r-\r-\r47,22 (2)\r16,22\r41,38\r104,82\..."
2,"Alagoas\r(alíquota de ICMS do produtor)\r18,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"43,88\r-\r-\r43,88 (2)\r13,88\r15,90\r73,66\r...","44,82\r-\r-\r44,82 (2)\r13,88\r14,71\r73,41\r..."
3,"Amapá\r(alíquota de ICMS do produtor)\r18,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"48,31\r-\r-\r48,31 (2)\r16,64\r19,02\r83,97\r...","47,22\r-\r-\r47,22 (2)\r16,78\r20,34\r84,34\r..."
0,"Amazonas\r(alíquota de ICMS do produtor)\r18,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"56,01\r-\r-\r56,01\r15,40\r39,79\r111,20\r11,0...","55,58\r-\r-\r55,58\r15,60\r42,00\r113,18\r10,8..."


In [58]:
#EXPLORING THE (margens_23) DATAFRAME:
margens_23.tail()

,Unnamed: 0,Unnamed: 1,jan/23,fev/23
3,"Roraima\r(alíquota de ICMS do produtor)\r12,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"48,31\r-\r-\r48,31 (2)\r12,81\r56,69\r117,81\...","47,22\r-\r-\r47,22 (2)\r14,13\r58,41\r119,76\..."
0,Santa Catarina\r(alíquota de ICMS do produtor)...,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"43,50\r-\r-\r43,50\r20,73\r16,11\r80,34\r42,45...","43,50\r-\r-\r43,50\r20,86\r16,07\r80,43\r43,38..."
1,"São Paulo\r(alíquota de ICMS do produtor)\r12,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"41,89\r-\r-\r41,89\r13,60\r19,24\r74,73\r32,38...","41,79\r-\r-\r41,79\r12,87\r19,25\r73,91\r32,56..."
2,"Sergipe\r(alíquota de ICMS do produtor)\r12,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"43,88\r-\r-\r43,88 (2)\r10,57\r20,35\r74,80\r...","44,82\r-\r-\r44,82 (2)\r9,74\r19,47\r74,03\r2..."
0,"Tocantins\r(alíquota de ICMS do produtor)\r12,00%",Preço de Realização do Produtor\rCIDE\rPIS/COF...,"48,31\r-\r-\r48,31 (2)\r10,92\r18,51\r77,74\r...","47,22\r-\r-\r47,22 (2)\r12,01\r18,55\r77,77\r..."


In [59]:
#EXPLORING THE (margens_23) DATAFRAME:
margens_23.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 0
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  28 non-null     object
 1   Unnamed: 1  28 non-null     object
 2   jan/23      28 non-null     object
 3   fev/23      28 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


In [60]:
# DEFINING THE FUNCTION THAT EXTRACTS BRAZIL ONLY FROM THE GROUP.
def brasil_icms_23(df):
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE 'margens_23' DATAFRAME, AIMED AT RETURNING GENERAL DATA FOR BRAZIL.'''

# GENERAL TREATMENT:

    df = df.iloc[0:1]
    df = df.rename(columns = {'Unnamed: 1': 'ICMS em %', 'Unnamed: 0':'ESTADO'})
    df['ICMS em %']= df['ICMS em %'].str.replace('Preço de Realização do Produtor\rCIDE\rPIS/COFINS\rPreço do Produtor s/ ICMS c/ CIDE/PIS/COFINS\rICMS\rMargem Bruta de Distribuição\rPreço de Distribuição\rMargem Bruta de Revenda\rPreço Final ao Consumidor', '-')
    df = df.set_index('ESTADO')

#'jan/23' COLUMN TREATMENT:

    df['jan/23'] = df['jan/23'].str.slice(start=16)
    df['jan/23'] = df['jan/23'].str.slice(start=0, stop=5)
    df['jan/23'] = df['jan/23'].str.replace(',','.')

#'fev/23' COLUMN TREATMENT:

    df['fev/23'] = df['fev/23'].str.slice(start=16)
    df['fev/23'] = df['fev/23'].str.slice(start=0, stop=5)
    df['fev/23'] = df['fev/23'].str.replace(',','.')

    return df

In [61]:
# NEW DATAFRAME TREATED (br_icms_23):

br_icms_23 = brasil_icms_23(margens_23)

In [62]:
def limpar_margens_23(df):
   
    '''THIS FUNCTION USES THE 'margins_23' DATAFRAME TO CLEAN AND PROCESS GENERAL DATA WITH THE PURPOSE OF EXTRACTING THE ICMS FROM EACH STATE.'''

# GENERAL TREATMENT:
    
    df = df.drop(index=[0])
    df = df.rename(columns = {'Unnamed: 1': 'ICMS em %', 'Unnamed: 0':'ESTADO'})
    df['ICMS em %']= df['ICMS em %'].str.replace('Preço de Realização do Produtor\rCIDE\rPIS/COFINS\rPreço do Produtor s/ ICMS c/ CIDE/PIS/COFINS\rICMS\rMargem Bruta de Distribuição\rPreço de Distribuição\rMargem Bruta de Revenda\rPreço Final ao Consumidor', '-')
    lista_23 = ['AC','AL','AP','BA','CE','DF','G0','MA','MT','MG','PA','PB','PE','PI','RJ','RS','RO','RR','SP','SE']
    icms_lista_23 = ['17%','18%','18%','12%','18%','12%','12%','14%','12%','18%','17%','18%','18%','12%','12%','12%','12%','12%','12%','12%']
    df['ESTADO'] = lista_23
    df['ICMS em %'] = icms_lista_23 
    df = df.set_index('ESTADO')


#'jan/23' COLUMN TREATMENT:

    df['jan/23'] = df['jan/23'].str.slice(start=16, stop=27)
    lj = ['16,09','13,88','16,64','9,19','14,32','13,31','11,62','11,52','13,37','14,84','14,89','14,86','13,55','10,69','9,75','9,67','15,07','12,81','13,60','10,57']
    df['jan/23'] = lj
    df['jan/23'] = df['jan/23'].str.replace(',','.')


#'fev/23' COLUMN TREATMENT:
    
    df['fev/23'] = df['fev/23'].str.slice(start=16, stop=27)
    lf = ['16,22','13,88','16,78','19,37','15,02','13,44','13,37','13,49','14,45','14,97','15,02','15,91','13,67','10,69','10,91','9,75','14,63','14,13','12,87','9,74']
    df['fev/23'] = lf
    df['fev/23'] = df['fev/23'].str.replace(',','.')


    return df

In [63]:
# NEW DATAFRAME TREATED (br_icms_23):


estados_a =limpar_margens_23(margens_23)

*CONCATENATING THE ICMS TABLES BY STATE IN A NEW DATAFRAME*

In [64]:
# NEW DATAFRAME TREATED (estados_icms_2023):

estados_icms_2023 = pd.concat([estados_a, estados_complemento], axis=0)

***CREATING THE DATABASE IN MYSQL AND EXPORTING TABLES.***